Displaying a simple GIS map using Folium


In [59]:
import geopandas as gpd
import pandas as pd
import folium
from folium.features import GeoJsonTooltip

# Load the shapefile
gdf = gpd.read_file('../shapefiles/cb_2018_us_county_500k.shp')

# Load the population data
population_data = pd.read_csv('../data/Population_Change_County.csv')

# Merge the GeoDataFrame with the population data DataFrame on the common identifier
merged_gdf = gdf.merge(population_data, left_on='AFFGEOID', right_on='Geography')
merged_gdf

# Convert the merged GeoDataFrame to GeoJSON
geojson_data = merged_gdf.to_json()

# Create a base map
initial_coordinates = [37.0902, -95.7129]  # Coordinates of the geographic center of the US
mymap = folium.Map(location=initial_coordinates, zoom_start=4)

def create_choropleth(geojson_data, data, column, name, color):
    layer = folium.Choropleth(
        geo_data=geojson_data,
        data=data,
        columns=['AFFGEOID', column],
        key_on='feature.properties.AFFGEOID',
        fill_color=color,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=name + ' Population'
    )
    # Add tooltip
    tooltip = GeoJsonTooltip(
        fields=['NAME', column],
        aliases=['County:', 'Population:'],
        localize=True
    )
    layer.geojson.add_child(tooltip)
    return layer

# Add choropleth layers for the year 2000 and 2020
layer_2000 = create_choropleth(geojson_data, merged_gdf, 'Sum of Population2000', '2000', 'BuGn')
layer_2020 = create_choropleth(geojson_data, merged_gdf, 'Sum of Population2020', '2020', 'OrRd')

# Add layers to the map
layer_2000.add_to(mymap)
layer_2020.add_to(mymap)

# Add a layer control panel to the map
folium.LayerControl().add_to(mymap)

# Save the map to an HTML file
html_save_path = '../output/both_years_map.html'
mymap.save(html_save_path)

# Output the path to the saved HTML file
html_save_path


'../output/both_years_map.html'

In [60]:
import geopandas as gpd
import pandas as pd
import folium
from folium.features import GeoJsonTooltip

# Load the shapefile and population data, and merge them (assuming you have done this part)

# Create a base map
initial_coordinates = [37.0902, -89.7129]  # Coordinates of the geographic center of the US
map = folium.Map(location=initial_coordinates, zoom_start=4)

def create_choropleth(geojson_data, data, column, name, color):
    layer = folium.Choropleth(
        geo_data=geojson_data,
        data=data,
        columns=['AFFGEOID', column],
        key_on='feature.properties.AFFGEOID',
        fill_color=color,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=name + ' Percentage Difference'
    )
    # Add tooltip
    tooltip = GeoJsonTooltip(
        fields=['NAME', column],
        aliases=['County:', 'Percentage Difference:'],
        localize=True
    )
    layer.geojson.add_child(tooltip)
    return layer

layer_difference = create_choropleth(geojson_data, merged_gdf, 'Sum of Percentage Difference', 'Percentage Difference', 'BuGn')

# Add layers to the map
layer_difference.add_to(map)

# Add a layer control panel to the map
folium.LayerControl().add_to(map)

# Save the map to an HTML file
html_save_path = '../output/difference_map.html'
map.save(html_save_path)

# Output the path to the saved HTML file
html_save_path


'../output/difference_map.html'

In [61]:
import folium
import branca.colormap as cm

# Assuming geojson_data and merged_gdf are already defined and merged_gdf includes a column 'Sum of Percentage Difference'

# Create a base map
initial_coordinates = [37.0902, -89.7129]  # Coordinates of the geographic center of the US
map = folium.Map(location=initial_coordinates, zoom_start=4)

# Function to determine the color based on the percentage difference
def get_color(feature):
    value = feature['properties']['Sum of Percentage Difference']
    if value >= 0:
        return 'blue'  # Positive difference
    else:
        return 'red'   # Negative difference

# Custom style function using the color function
def style_function(feature):
    return {
        'fillColor': get_color(feature),
        'color': 'black',  # Boundary color
        'weight': 1,
        'fillOpacity': 0.7
    }

# Tooltip fields
tooltip = GeoJsonTooltip(
    fields=['NAME', 'Sum of Percentage Difference'],
    aliases=['County:', 'Percentage Difference:'],
    localize=True
)

# Add the GeoJson layer to the map
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=tooltip
).add_to(map)

# Add a layer control panel to the map
folium.LayerControl().add_to(map)

# Save the map to an HTML file
html_save_path = '../output/difference_map.html'
map.save(html_save_path)

# Output the path to the saved HTML file
html_save_path


'../output/difference_map.html'

In [62]:
import folium
import branca.colormap as cm
import pandas as pd
import geopandas as gpd
from folium.features import GeoJsonTooltip

# Load the shapefile and population data, and merge them
gdf = gpd.read_file('../shapefiles/cb_2018_us_county_500k.shp')
population_data = pd.read_csv('../data/Population_Change_County.csv')
merged_gdf = gdf.merge(population_data, left_on='AFFGEOID', right_on='Geography')

# Convert the merged GeoDataFrame to GeoJSON
geojson_data = merged_gdf.to_json()

# Create a base map
initial_coordinates = [37.0902, -95.7129]  # Coordinates of the geographic center of the US
map = folium.Map(location=initial_coordinates, zoom_start=4)

# Define a continuous colormap
min_val = merged_gdf['Sum of Percentage Difference'].min()
max_val = merged_gdf['Sum of Percentage Difference'].max()
colormap = cm.LinearColormap(colors=['red', 'white', 'blue'], vmin=min_val, vmax=max_val)

# Custom style function
def style_function(feature):
    value = feature['properties']['Sum of Percentage Difference']
    return {
        'fillColor': colormap(value),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# Add tooltip
tooltip = GeoJsonTooltip(
    fields=['NAME', 'Sum of Percentage Difference'],
    aliases=['County:', 'Percentage Difference:'],
    localize=True
)

# Add the GeoJson layer to the map
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=tooltip
).add_to(map)

# Add the colormap to the map
colormap.caption = 'Percentage Difference'
colormap.add_to(map)

# Add a layer control panel to the map
folium.LayerControl().add_to(map)

# Save the map to an HTML file
html_save_path = '../output/difference_map.html'
map.save(html_save_path)

# Output the path to the saved HTML file
html_save_path


'../output/difference_map.html'

In [63]:
import folium
import branca.colormap as cm
import pandas as pd
import geopandas as gpd
from folium.features import GeoJsonTooltip

# Load the shapefile and population data, and merge them
gdf = gpd.read_file('../shapefiles/cb_2018_us_county_500k.shp')
population_data = pd.read_csv('../data/Population_Change_County.csv')
merged_gdf = gdf.merge(population_data, left_on='AFFGEOID', right_on='Geography')

# Convert the merged GeoDataFrame to GeoJSON
geojson_data = merged_gdf.to_json()

# Create a base map
initial_coordinates = [37.0902, -95.7129]  # Coordinates of the geographic center of the US
map = folium.Map(location=initial_coordinates, zoom_start=4)

# Define a symmetric colormap
max_abs_val = max(abs(merged_gdf['Sum of Percentage Difference'].min()), abs(merged_gdf['Sum of Percentage Difference'].max()))
colormap = cm.LinearColormap(colors=['red', 'white', 'blue'], vmin=-max_abs_val, vmax=max_abs_val)

# Custom style function
def style_function(feature):
    value = feature['properties']['Sum of Percentage Difference']
    return {
        'fillColor': colormap(value),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# Add tooltip
tooltip = GeoJsonTooltip(
    fields=['NAME', 'Sum of Percentage Difference'],
    aliases=['County:', 'Percentage Difference:'],
    localize=True
)

# Add the GeoJson layer to the map
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=tooltip
).add_to(map)

# Add the colormap to the map
colormap.caption = 'Percentage Difference'
colormap.add_to(map)

# Add a layer control panel to the map
folium.LayerControl().add_to(map)

# Save the map to an HTML file
html_save_path = '../output/difference_map.html'
map.save(html_save_path)

# Output the path to the saved HTML file
html_save_path


'../output/difference_map.html'